In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from sklearn import preprocessing

In [4]:
training_data = pd.read_csv("/home/roboto/Documents/GitHub/tutorials/data/weather/training.csv", sep = "\t")
testing_data  = pd.read_csv("/home/roboto/Documents/GitHub/tutorials/data/weather/testing.csv", sep = ",")
# l1, l2 = len(training_data), len(testing_data)
# min_max_scaler = preprocessing.MinMaxScaler()
# training_data['Temperature'] = min_max_scaler.fit_transform(training_data['Temperature'].values.reshape(l1, -1))
# testing_data['Temperature'] = min_max_scaler.fit_transform(testing_data['Temperature'].values.reshape(l2, -1))
print("Training")
print(training_data.head())
print("Testing")
print(testing_data.head())

Training
         Date  Temperature
0  01/01/2011         68.9
1  01/02/2011         66.4
2  01/03/2011         68.7
3  01/04/2011         71.4
4  01/05/2011         69.3
Testing
         Date  Temperature
0  09/03/2019         73.4
1  09/04/2019         71.6
2  09/05/2019         71.7
3  09/06/2019         71.6
4  09/07/2019         71.3


In [62]:
class WeatherTemperatureDataset(Dataset):
    def __init__(self, dataframe):
        super(WeatherTemperatureDataset, self).__init__()
        self.samples = []
        self.generate_samples(dataframe)
        
    def generate_samples(self, dataframe):
        series = dataframe['Temperature']
        n_days = 1
        for i in range(30, len(series) - n_days, 30):
            _input = torch.tensor(series[i - 30: i].values, dtype = torch.float)
            if True in torch.isnan(_input):
                pass
            else:
                _input = _input.reshape(-1, 1)
                target = torch.tensor(series[i:i+n_days].values, dtype = torch.float)
                self.samples.append((_input, target))
        
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        return self.samples[idx]
               

In [63]:
training_dataset = WeatherTemperatureDataset(training_data)
testing_dataset  = WeatherTemperatureDataset(training_data)

In [70]:
batch_size = 3
training_dataloader = DataLoader(training_dataset, shuffle = True, batch_size = batch_size, drop_last = True)
testing_dataloader = DataLoader(testing_dataset, shuffle = True)
sample = next(iter(training_dataloader))
input_sample = sample[0]

***

### RNN Structures

__Basic Module__

From this module, it's possible to build three basic types of architectures:

- Recurrent networks that produce an output at each time step and have recurrent connections between hidden units
- Recurrent networks that produce an output at each time step and have recurrent connections only from the output at one time step to the hidden units at the next time step
- Recurrent networks with recurrent connections between hidden units, that read an entire sequence and then produce a single output

In [106]:
class RNNModule(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, hidden_activation = 'tanh', output_activation = 'sigmoid'):
        super().__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.output_size = output_size 
        
        self.hidden_linear = nn.Linear(self.hidden_size, self.hidden_size)
        self.input_linear  = nn.Linear(self.input_size, self.hidden_size)
        self.output_linear = nn.Linear(self.hidden_size, self.output_size)
        
        self.hidden_activation = nn.functional.tanh
        self.output_activation = nn.functional.sigmoid # Put a selection function here
        
        self.first = True # if True, the network wasn't used before
        
    def forward(self, X, Y: torch.Tensor = None):
        '''
        X(n_batches, sequence_length, num_features)
        '''
        if self.first:
            self.initHidden()
            self.first = False
        
        output = torch.cat(self.sequence_processing(X, Y), 1)
        return output, self.hidden_state
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_size, dtype = torch.float)
    
    def reset(self):
        self.first = True
        
    def sequence_processing(self, X: torch.Tensor, Y: torch.Tensor = None) -> List[torch.Tensor]:
        raise NotImplementedError("Each subclass of a recurrent module class should implement it's own method to compute outputs!")

![RNN_Saved_Hidden](Images/RNN_Saved_Hidden.jpg)

In [ ]:
class RNNBasic(RNNModule):
    # Returns one output for each sequence input, i.e. returns another whole sequence
    # Each state is computed using the immidiate input and the previous hidden state.
    def __init__(self, input_size, hidden_size, output_size, activation = 'tanh', output_activation = 'sigmoid'):
        super().__init__(input_size, hidden_size, output_size, activation)
        
    def sequence_processing(self, X: torch.Tensor, Y: torch.Tensor = None) -> List[torch.Tensor]:
        output = []
        for i in range(X.shape[1]):
            self.hidden_state = self.hidden_activation(self.input_linear(X[:, i, :]) + self.hidden_linear(self.hidden_state))
            output.append(self.output_activation(self.output_linear(self.hidden_state)).unsqueeze(1))
        
        return output

![RNN_Saved_Output](Images/RNN_Saved_Output.jpg)

_Teacher Forcing_

Instead of the previous output, the previous target valued is used to compute the immidiate hidden state.

![RNN_Teacher_Forcing](Images/RNN_Teacher_Forcing.jpg)

In [ ]:
class RNNBasic_OutputRecurrence(RNNModule):
    # Returns one output for each sequence input, i.e. returns another whole sequence
    # Each state is computed using the immidiate input and the previous output
    def __init__(self, input_size, hidden_size, output_size, activation = 'tanh', output_activation = 'sigmoid'):
        super().__init__(input_size, output_size, output_size, activation)
        # the previous hidden state is the saved previous 
        
    def sequence_processing(self, X: torch.Tensor, Y: torch.Tensor = None) -> List[torch.Tensor]:
        output = []
        for i in range(X.shape[1]):
            hidden_state = self.hidden_activation(self.input_linear(X[:, i, :]) + self.hidden_linear(self.hidden_state))
            output.append(self.output_activation(self.output_linear(self.hidden_state)).unsqueeze(1))
            self.hidden_state = output[-1]
        
        return output

![RNN_Single_Output](Images/RNN_Single_Output.jpg)

In [ ]:
class RNNBasic_SingleOutput(RNNModule):
    # Returns one output for each sequence input, i.e. returns another whole sequence
    # Each state is computed using the immidiate input and the previous hidden state.
    def __init__(self, input_size, hidden_size, output_size, activation = 'tanh'):
        super().__init__(input_size, hidden_size, output_size, activation)
        
    def sequence_processing(self, X: torch.Tensor, Y: torch.Tensor = None) -> List[torch.Tensor]:
        for i in range(X.shape[1]):
            self.hidden_state = self.hidden_activation(self.input_linear(X[:, i, :]) + self.hidden_linear(self.hidden_state))
        
        output = self.output_activation(self.output_linear(self.hidden_state)).unsqueeze(1)
        
        return output

***

In [ ]:
# class RNN(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers, n_days):
#         super(RNN, self).__init__()
#         self.hidden_size = hidden_size
#         self.n_layers = num_layers
#         self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first = True)
#         self.output = nn.Linear(hidden_size, n_days)
                            
        
#     def forward(self, x, hidden):
#         out, hidden = self.rnn(x, hidden)
#         out = hidden[-1]
#         out = self.output(out)
        
#         return out, hidden
    
# def train_rnn(model, dataloader, loss_function, optimizer, epochs, scheduler = None):
#     epoch_losses = []
#     mean = lambda xs : sum(xs)/len(xs) 
#     for i in range(1, epochs+1):
#         temp_losses = []
#         for sample in dataloader: 
#                 optimizer.zero_grad()
#                 hidden = None
                
#                 inputs = sample[0]
#                 targets = sample[1]
#                 output, hidden = model(inputs, hidden)
                
#                 loss = loss_function(output, targets)
#                 temp_losses.append(loss.item())
# #                 print(loss)
#                 loss.backward(retain_graph = True)
#                 optimizer.step()
                
#         if scheduler != None:
#             scheduler.step()
            
#         epoch_losses.append(mean(temp_losses))
#         if i%10 == 0:
#             print('Epoch: {}/{}.............'.format(i, epochs), end=' ')
#             print("Loss: {:.4f}".format(epoch_losses[-1]))
    
#     return epoch_losses